<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study2/blob/main/faultDetector1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical

In [44]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_data'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [45]:
# 모델 정의
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))  # 드롭아웃 비율 0.5
    model.add(Dense(len(categories), activation='softmax'))
    return model

model = create_model()

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 및 학습률 감소 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)


In [46]:
# 모델 학습
history = model.fit(X_train, y_train_categorical, epochs=20, batch_size=32, validation_data=(X_val, y_val_categorical), callbacks=[early_stopping, reduce_lr])

Epoch 1/20
219/219 [==============================] - 144s 651ms/step - loss: 1.4477 - accuracy: 0.5086 - val_loss: 0.6063 - val_accuracy: 0.7246 - lr: 0.0010
Epoch 2/20
219/219 [==============================] - 144s 658ms/step - loss: 0.6328 - accuracy: 0.7220 - val_loss: 0.5031 - val_accuracy: 0.7246 - lr: 0.0010
Epoch 3/20
219/219 [==============================] - 146s 665ms/step - loss: 0.6051 - accuracy: 0.6759 - val_loss: 0.2232 - val_accuracy: 0.8576 - lr: 0.0010
Epoch 4/20
219/219 [==============================] - 148s 675ms/step - loss: 0.4455 - accuracy: 0.7610 - val_loss: 0.1685 - val_accuracy: 0.9397 - lr: 0.0010
Epoch 5/20
219/219 [==============================] - 149s 680ms/step - loss: 0.4014 - accuracy: 0.7983 - val_loss: 0.1532 - val_accuracy: 0.9945 - lr: 0.0010
Epoch 6/20
219/219 [==============================] - 149s 683ms/step - loss: 0.3919 - accuracy: 0.8117 - val_loss: 0.1269 - val_accuracy: 0.9961 - lr: 0.0010
Epoch 7/20
219/219 [==========================

In [47]:
# 모델 사이즈 확인
model.save('model.h5')
model_size = os.path.getsize('model.h5') / (1024 * 1024)  # 모델 사이즈를 MB 단위로 변환
print(f"Model Size: {model_size:.2f} MB")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model Size: 1124.98 MB


In [48]:
# 검증 데이터 평가
val_loss, val_accuracy = model.evaluate(X_val, y_val_categorical)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

40/40 [==============================] - 5s 112ms/step - loss: 0.0226 - accuracy: 1.0000
Validation Loss: 0.022570721805095673
Validation Accuracy: 1.0


In [49]:
# 테스트 데이터 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

40/40 [==============================] - 4s 108ms/step - loss: 0.0300 - accuracy: 0.9969
Test Loss: 0.029992295429110527
Test Accuracy: 0.9968701004981995


In [50]:
# 혼동 행렬 및 성능 지표 출력 (검증 데이터)
y_pred_val = model.predict(X_val)
y_pred_val_classes = np.argmax(y_pred_val, axis=1)

conf_matrix_val = confusion_matrix(y_val_encoded, y_pred_val_classes)
class_report_val = classification_report(y_val_encoded, y_pred_val_classes, target_names=categories)

print("검증 데이터에 대한 혼동 행렬:")
print(conf_matrix_val)
print("\n검증 데이터에 대한 분류 보고서:")
print(class_report_val)

40/40 [==============================] - 4s 106ms/step
검증 데이터에 대한 혼동 행렬:
[[176   0   0   0]
 [  0 176   0   0]
 [  0   0 176   0]
 [  0   0   0 750]]

검증 데이터에 대한 분류 보고서:
              precision    recall  f1-score   support

      normal       1.00      1.00      1.00       176
    fault_BB       1.00      1.00      1.00       176
    fault_RI       1.00      1.00      1.00       176
    fault_SM       1.00      1.00      1.00       750

    accuracy                           1.00      1278
   macro avg       1.00      1.00      1.00      1278
weighted avg       1.00      1.00      1.00      1278



In [51]:
# 혼동 행렬 및 성능 지표 출력 (테스트 데이터)
y_pred_test = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

conf_matrix_test = confusion_matrix(y_test_encoded, y_pred_test_classes)
class_report_test = classification_report(y_test_encoded, y_pred_test_classes, target_names=categories)

print("테스트 데이터에 대한 혼동 행렬:")
print(conf_matrix_test)
print("\n테스트 데이터에 대한 분류 보고서:")
print(class_report_test)

40/40 [==============================] - 4s 104ms/step
테스트 데이터에 대한 혼동 행렬:
[[172   0   4   0]
 [  0 176   0   0]
 [  0   0 176   0]
 [  0   0   0 750]]

테스트 데이터에 대한 분류 보고서:
              precision    recall  f1-score   support

      normal       1.00      0.98      0.99       176
    fault_BB       1.00      1.00      1.00       176
    fault_RI       0.98      1.00      0.99       176
    fault_SM       1.00      1.00      1.00       750

    accuracy                           1.00      1278
   macro avg       0.99      0.99      0.99      1278
weighted avg       1.00      1.00      1.00      1278

